# EBSI Wallet Conformance (Natural Person)

> EBSI v2 DID for Natural Person(s) is introduced. 

## Setup conformance headers

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

headers = {
    "Conformance": str(uuid.uuid4()),
}

## Setup wallet

In [2]:
from ebsi_wallet.main import wallet

# Setup wallet
client = await wallet("init_v2")


## ISSUE_011

In [3]:
from ebsi_wallet.main import conformance
from ebsi_wallet.util import parse_query_string_parameters_from_url

result = await conformance("issuerInitiate", headers=headers)

console.log("issuerInitiate", result)

session_token = result.replace("openid://", "")
credential_type = parse_query_string_parameters_from_url(
    session_token).get("credential_type")[0]
issuer_url = parse_query_string_parameters_from_url(
    session_token).get("issuer")[0]

console.log("credential_type", credential_type)

options = {
    "client": client,
    "credential_type" : credential_type
}

result = await conformance("issuerAuthorize", options=options)
console.log("issuerAuthorize", result)


[15:57:45] issuerInitiate                                                                           ]8;id=297506;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py\4201365539.py]8;;\:]8;id=452484;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py#6\6]8;;\
           openid://initiate_issuance?issuer=https%3A%2F%2Fapi.conformance.intebsi.xyz%2Fconformanc                
           e%2Fv2&credential_type=https%3A%2F%2Fapi.conformance.intebsi.xyz%2Ftrusted-schemas-regis                
           try%2Fv2%2Fschemas%2FzCfNxx5dMBdf4yVcsWzj1anWRuXcxrXj1aogyfN1xSu8t&conformance=788d791d-                
           651b-49ca-827d-e984f0875382                                                                             

           credential_type                                                                         ]8;id=514967;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py\4201365539.py]8;;\:]8;id=887102;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py#14\14]8;;\
           https://api.conformance.intebsi.xyz/trusted-schemas-registry/v2/schemas/zCfNxx5dMBdf4yV                 
           csWzj1anWRuXcxrXj1aogyfN1xSu8t                                                                          

           issuerAuthorize                                                                         ]8;id=362090;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py\4201365539.py]8;;\:]8;id=411706;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/4201365539.py#22\22]8;;\
           {'state': '98a4a4a0a0ba', 'code': 'fb67fe00cca562b65e60'}                                               

## ISSUE_021

In [4]:
options = {
    "did_version": client.did_version,
    "code": result["code"]
}

result = await conformance("issuerToken", options=options, headers=headers)
console.log("issuerToken", result)


[15:57:46] issuerToken                                                                              ]8;id=870236;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/1256846934.py\1256846934.py]8;;\:]8;id=324859;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/1256846934.py#7\7]8;;\
           {                                                                                                       
               'token_type': 'Bearer',                                                                             
               'access_token': 'da019bfbd503df0c9881',                                                             
               'expires_in': 3600,                                                                                 
               'c_nonce': 'eb106aa8966de772e000',                                                                  
               'id_token':                                                                                         
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpYXQiOjE2NjYzNDgwNjYsInN1YiI6Imh0dHBzOi8vbG9j                
           YWxob3N0OjMwMDAiLCJhdWQiOiJodHRwczovL2xvY2FsaG9zdDozMDAwIiwiaXNzIjoiZGlkOmVic2k6emNmY3dH                
           akxCb2pjenc5eWhtVUZFM1oifQ.K4ojf5jGIaiku0KKnXmF1-5t2Wt0CCzI0OjmXI4Dc-GzRmyN2nGg_UoZqE52P                
           atfTbSmtXVrEjeZRLXnW3XQJA'                                                                              
           }                                                                                                       

## ISSUE_031

In [5]:
from ebsi_wallet.did_jwt import decode_jwt

options = {
    "c_nonce": result["c_nonce"],
    "client": client,
    "issuer_url": issuer_url,
    "credential_type": credential_type,
}

access_token = result["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}",
    "Conformance": headers["Conformance"]
}

result = await conformance("issuerCredential", options=options, headers=headers)
console.log("issuerCredential", result)

issue_031_credential = result["credential"]

credential = decode_jwt(result["credential"])

console.log("Received credential", credential)

[15:57:47] issuerCredential                                                                         ]8;id=499058;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/608560569.py\608560569.py]8;;\:]8;id=404486;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/608560569.py#18\18]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NksiLCJ0eXAiOiJKV1QiLCJraWQiOiJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkU                
           zWiNrZXlzLTEifQ.eyJqdGkiOiJ1cm46ZGlkOmQyNzQwYmYzLWU1MGYtNDNjZi1hOGU4LTY5MmUxOWY1Mzc1NyIs                
           InN1YiI6ImRpZDplYnNpOnpyRmRZQlMzS3g1WFFFcGFGMjE0aWM3Wlgza1FRNFRwZXVKa0xZV2VNdVU1WiIsImlz                
           cyI6ImRpZDplYnNpOnpjZmN3R2pMQm9qY3p3OXlobVVGRTNaIiwibmJmIjoxNjY2MzQ4MDY3LCJpYXQiOjE2NjYz                
           NDgwNjcsInZjIjp7IkBjb250ZXh0IjpbImh0dHBzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0s                
           ImlkIjoidXJuOmRpZDpkMjc0MGJmMy1lNTBmLTQzY2YtYThlOC02OTJlMTlmNTM3NTciLCJ0eXBlIjpbIlZlcmlm                
           aWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiVmVyaWZpYWJsZUlkIl0sImlzc3VlciI6                
           ImRpZDplYnNpOnpjZmN3R2pMQm9qY3p3OXlobVVGRTNaIiwiaXNzdWFuY2VEYXRlIjoiMjAyMi0xMC0yMVQxMDoy                
           Nzo0N1oiLCJ2YWxpZEZyb20iOiIyMDIyLTEwLTIxVDEwOjI3OjQ3WiIsImlzc3VlZCI6IjIwMjItMTAtMjFUMTA6                
           Mjc6NDdaIiwiY3JlZGVudGlhbFN1YmplY3QiOnsiaWQiOiJkaWQ6ZWJzaTp6ckZkWUJTM0t4NVhRRXBhRjIxNGlj                
           N1pYM2tRUTRUcGV1SmtMWVdlTXVVNVoiLCJwZXJzb25hbElkZW50aWZpZXIiOiJJVC9ERS8xMjM0IiwiZmFtaWx5                
           TmFtZSI6IkNhc3RhZmlvcmkiLCJmaXJzdE5hbWUiOiJCaWFuY2EiLCJkYXRlT2ZCaXJ0aCI6IjE5MzAtMTAtMDEi                
           fSwiY3JlZGVudGlhbFNjaGVtYSI6eyJpZCI6Imh0dHBzOi8vYXBpLmNvbmZvcm1hbmNlLmludGVic2kueHl6L3Ry                
           dXN0ZWQtc2NoZW1hcy1yZWdpc3RyeS92Mi9zY2hlbWFzL3pDZk54eDVkTUJkZjR5VmNzV3pqMWFuV1J1WGN4clhq                
           MWFvZ3lmTjF4U3U4dCIsInR5cGUiOiJGdWxsSnNvblNjaGVtYVZhbGlkYXRvcjIwMjEifX19.BIOSSRrGb5IO0yY                
           G3aqHrrBWupXFa-vFheccP9PQ5JQ7j5gvTfgbm47P3w5gh2ml033DGMeLGO-5pVU3MYndTg',                               
               'c_nonce': '6113cdd474615c862a3c',                                                                  
               'c_nonce_expires_in': 900                                                                           
           }                                                                                                       

           Received credential                                                                      ]8;id=7661;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/608560569.py\608560569.py]8;;\:]8;id=329587;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/608560569.py#24\24]8;;\
           {                                                                                                       
               'header': {                                                                                         
                   'alg': 'ES256K',                                                                                
                   'typ': 'JWT',                                                                                   
                   'kid': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z#keys-1'                                                
               },                                                                                                  
               'payload': {                                                                                        
                   'jti': 'urn:did:d2740bf3-e50f-43cf-a8e8-692e19f53757',                                          
                   'sub': 'did:ebsi:zrFdYBS3Kx5XQEpaF214ic7ZX3kQQ4TpeuJkLYWeMuU5Z',                                
                   'iss': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                                      
                   'nbf': 1666348067,                                                                              
                   'iat': 1666348067,                                                                              
                   'vc': {                                                                                         
                       '@context': ['https://www.w3.org/2018/credentials/v1'],                                     
                       'id': 'urn:did:d2740bf3-e50f-43cf-a8e8-692e19f53757',                                       
                       'type': ['VerifiableCredential', 'VerifiableAttestation', 'VerifiableId'],                  
                       'issuer': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                               
                       'issuanceDate': '2022-10-21T10:27:47Z',                                                     
                       'validFrom': '2022-10-21T10:27:47Z',                                                        
                       'issued': '2022-10-21T10:27:47Z',                                                           
                       'credentialSubject': {                                                                      
                           'id': 'did:ebsi:zrFdYBS3Kx5XQEpaF214ic7ZX3kQQ4TpeuJkLYWeMuU5Z',                         
                           'personalIdentifier': 'IT/DE/1234',                                                     
                           'familyName': 'Castafiori',                                                             
                           'firstName': 'Bianca',                                                                  
                           'dateOfBirth': '1930-10-01'                                                             
                       },                                                                                          
                       'credentialSchema': {                                                                       
                           'id':                                                                                   
           'https://api.conformance.intebsi.xyz/trusted-schemas-registry/v2/schemas/zCfNxx5dMBdf4yV                
           csWzj1anWRuXcxrXj1aogyfN1xSu8t',                                                                        
                           'type': 'FullJsonSchemaValidator2021'         

## VERIFY_011

In [6]:
headers = {
    "Conformance": headers.get("Conformance")
}
options = {
    "did_version": "v2"
}

result = await conformance("verifierAuthRequest", options=options, headers=headers)
console.log("verifierAuthRequest", result)

[15:57:48] verifierAuthRequest                                                                      ]8;id=542783;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/3364399777.py\3364399777.py]8;;\:]8;id=226134;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/3364399777.py#9\9]8;;\
           {                                                                                                       
               'client_id':                                                                                        
           'https://api.conformance.intebsi.xyz/conformance/v2/verifier-mock/authentication-respons                
           es',                                                                                                    
               'response_type': 'id_token',                                                                        
               'scope': 'openid',                                                                                  
               'claims':                                                                                           
           '{"id_token":{"email":null},"vp_token":{"presentation_definition":{"id":"conformance_moc                
           k_vp_request","input_descriptors":[{"id":"conformance_mock_vp","name":"Conformance Mock                 
           VP","purpose":"Only accept a VP containing a Conformance Mock                                           
           VA","constraints":{"fields":[{"path":["$.vc.credentialSchema"],"filter":{"allOf":[{"type                
           ":"array","contains":{"type":"object","properties":{"id":{"type":"string","pattern":"htt                
           ps://api.conformance.intebsi.xyz/trusted-schemas-registry/v2/schemas/z3kRpVjUFj4Bq8qHREN                
           UHiZrVF5VgMBUe7biEafp1wf2J"}},"required":["id"]}}]}}]}}],"format":{"jwt_vp":{"alg":["ES2                
           56K"]}}}}}',                                                                                            
               'redirect_uri':                                                                                     
           'https://api.conformance.intebsi.xyz/conformance/v2/verifier-mock/authentication-respons                
           es',                                                                                                    
               'nonce': 'aa83ea6f-3c67-46bd-8c95-74feff581797'                                                     
           }                                                                                                       

### Create Verifiable Presentation JWT

In [11]:
from ebsi_wallet.main import compute

encoded_credential = issue_031_credential
credential_issuer = credential['payload']['iss']

options = {
    "credential": encoded_credential,
    "client": client,
    "issuer": credential_issuer,
    "audience": credential_issuer
}

jwt_vp = await compute("createPresentationJwt", options=options)
console.log("createPresentationJwt", jwt_vp)

[15:57:57] createPresentationJwt                                                                   ]8;id=148526;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/2803278071.py\2803278071.py]8;;\:]8;id=278897;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/2803278071.py#14\14]8;;\
           {                                                                                                       
               'jwtVp':                                                                                            
           'eyJhbGciOiAiRVMyNTZLIiwgInR5cCI6ICJKV1QiLCAia2lkIjogImRpZDplYnNpOnpyRmRZQlMzS3g1WFFFcG                 
           FGMjE0aWM3Wlgza1FRNFRwZXVKa0xZV2VNdVU1WiMyOHdhbU8wYnhyc0tmRkJ5YTRzWnFELUpVVVJ5Vkd1SDNwc                 
           U5Zdm84bVJRIiwgImp3ayI6IHsia3R5IjogIkVDIiwgImNydiI6ICJzZWNwMjU2azEiLCAieCI6ICJtVVR5RnFw                 
           bWZIMlcyQ2dxNTNOY2lWaDhGVENWdUQ3bzZlbkFEV0F5bEdzIiwgInkiOiAiamg4M19ITUNLaDJ3S1dWdkl0TnB                 
           3ZHktS0tYZGUwdVF4eWpyUG0xem9OQSJ9fQ.eyJqdGkiOiAidXJuOmRpZDo3Y2U0ZDFkNi03MWNjLTRhNjctYjd                 
           mYy05YTBjMDM5N2I3N2UiLCAic3ViIjogImRpZDplYnNpOnpyRmRZQlMzS3g1WFFFcGFGMjE0aWM3Wlgza1FRNF                 
           RwZXVKa0xZV2VNdVU1WiIsICJpc3MiOiAiZGlkOmVic2k6enJGZFlCUzNLeDVYUUVwYUYyMTRpYzdaWDNrUVE0V                 
           HBldUprTFlXZU11VTVaIiwgIm5iZiI6IDE2NjYzNDgwNzAsICJleHAiOiAxNjY2MzQ4OTc3LCAiaWF0IjogMTY2                 
           NjM0ODA3NywgImF1ZCI6ICJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkUzWiIsICJ2cCI6IHsiaWQiOiA                 
           idXJuOmRpZDo3Y2U0ZDFkNi03MWNjLTRhNjctYjdmYy05YTBjMDM5N2I3N2UiLCAiQGNvbnRleHQiOiBbImh0dH                 
           BzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0sICJ0eXBlIjogWyJWZXJpZmlhYmxlUHJlc2Vud                 
           GF0aW9uIl0sICJob2xkZXIiOiAiZGlkOmVic2k6enJGZFlCUzNLeDVYUUVwYUYyMTRpYzdaWDNrUVE0VHBldUpr                 
           TFlXZU11VTVaIiwgInZlcmlmaWFibGVDcmVkZW50aWFsIjogWyJleUpoYkdjaU9pSkZVekkxTmtzaUxDSjBlWEF                 
           pT2lKS1YxUWlMQ0pyYVdRaU9pSmthV1E2WldKemFUcDZZMlpqZDBkcVRFSnZhbU42ZHpsNWFHMVZSa1V6V2lOcl                 
           pYbHpMVEVpZlEuZXlKcWRHa2lPaUoxY200NlpHbGtPbVF5TnpRd1ltWXpMV1UxTUdZdE5ETmpaaTFoT0dVNExUW                 
           TVNbVV4T1dZMU16YzFOeUlzSW5OMVlpSTZJbVJwWkRwbFluTnBPbnB5Um1SWlFsTXpTM2cxV0ZGRmNHRkdNakUw                 
           YVdNM1dsZ3phMUZSTkZSd1pYVkthMHhaVjJWTmRWVTFXaUlzSW1semN5STZJbVJwWkRwbFluTnBPbnBqWm1OM1I                 
           ycE1RbTlxWTNwM09YbG9iVlZHUlROYUlpd2libUptSWpveE5qWTJNelE0TURZM0xDSnBZWFFpT2pFMk5qWXpORG                 
           d3Tmpjc0luWmpJanA3SWtCamIyNTBaWGgwSWpwYkltaDBkSEJ6T2k4dmQzZDNMbmN6TG05eVp5OHlNREU0TDJOe                 
           VpXUmxiblJwWVd4ekwzWXhJbDBzSW1sa0lqb2lkWEp1T21ScFpEcGtNamMwTUdKbU15MWxOVEJtTFRRelkyWXRZ                 
           VGhsT0MwMk9USmxNVGxtTlRNM05UY2lMQ0owZVhCbElqcGJJbFpsY21sbWFXRmliR1ZEY21Wa1pXNTBhV0ZzSWl                 
           3aVZtVnlhV1pwWVdKc1pVRjBkR1Z6ZEdGMGFXOXVJaXdpVm1WeWFXWnBZV0pzWlVsa0lsMHNJbWx6YzNWbGNpST                 
           ZJbVJwWkRwbFluTnBPbnBqWm1OM1IycE1RbTlxWTNwM09YbG9iVlZHUlROYUlpd2lhWE56ZFdGdVkyVkVZWFJsS                 
           WpvaU1qQXlNaTB4TUMweU1WUXhNRG95TnpvME4xb2lMQ0oyWVd4cFpFWnliMjBpT2lJeU1ESXlMVEV3TFRJeFZE                 
           RXdPakkzT2pRM1dpSXNJbWx6YzNWbFpDSTZJakl3TWpJdE1UQXRNakZVTVRBNk1qYzZORGRhSWl3aVkzSmxaR1Z                 
           1ZEdsaGJGTjFZbXBsWTNRaU9uc2lhV1FpT2lKa2FXUTZaV0p6YVRwNmNrWmtXVUpUTTB0NE5WaFJSWEJoUmpJeE                 
           5HbGpOMXBZTTJ0UlVUUlVjR1YxU210TVdWZGxUWFZWTlZvaUxDSndaWEp6YjI1aGJFbGtaVzUwYVdacFpYSWlPa                 
           UpKVkM5RVJTOHhNak0wSWl3aVptRnRhV3g1VG1GdFpTSTZJa05oYzNSaFptbHZjbWtpTENKbWFYSnpkRTVoYldV                 
           aU9pSkNhV0Z1WTJFaUxDSmtZWFJsVDJaQ2FYSjBhQ0k2SWpFNU16QXRNVEF0TURFaWZTd2lZM0psWkdWdWRHbGh                 
           iRk5qYUdWdFlTSTZleUpwWkNJNkltaDBkSEJ6T2k4dllYQnBMbU52Ym1ad

## VERIFY_031

In [12]:
options = {
    "credential": encoded_credential,
    "jwtVp": jwt_vp["jwtVp"],
    "client": client
}

vp_status = await conformance("verifierAuthResponse", options=options, headers={})
console.log("Verifiable presentation status", vp_status)

[15:57:59] Verifiable presentation status                                                           ]8;id=993210;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/3868489859.py\3868489859.py]8;;\:]8;id=617392;file:///var/folders/z6/z5bp3zsx3g37nkmnvk3mp8880000gn/T/ipykernel_22288/3868489859.py#8\8]8;;\
           {                                                                                                       
               'result': True,                                                                                     
               'validations': {                                                                                    
                   'vpFormat': {'status': True},                                                                   
                   'presentation': {'status': True},                                                               
                   'credential': {'status': True}                                                                  
               }                                                                                                   
           }                                                                                                       